<a href="https://colab.research.google.com/github/FengruiJing/HIV_Prediction/blob/main/HIV_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Testing on Google Colab
https://www.youtube.com/watch?v=ULFlFznyUrc

In [4]:
#mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [6]:
#list the files
!ls

HIV_RNN.ipynb  TotalDataLast20.csv	 TotalDataTop20.csv
TotalData.csv  TotalDataSaveExample.csv


In [ ]:
!pip install howdoi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.5/312.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 35.9 MB/s eta 0:00:00
  Created wheel for howdoi: filename=howdoi-2.0.20-py3-none-any.whl size=16076 sha256=d43cb23f106c3dc17969eaaf1702357e57e0a3724c43746e37c4a177b3f2128f
  Stored in directory: /root/.cache/pip/wheels/31/a1/c8/80acaaa566a958b7dc3da1516e08ffd8caa45cfdd229b6ab8c
Successfully built howdoi
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handlin

In [ ]:

import pandas as pd
 
df = pd.read_csv("TotalDataTop20.csv")
print(df)
df.to_csv("TotalDataSaveExample.csv",index=False)

     County  Year  HIVprevalence     HIVXPCI1       HIVSCI1     HIVDis1
0     12125  2008         3395.5  3821045.108  1.630000e+11  357.155598
1     12125  2009         3521.6  3956743.323  1.690000e+11  368.891022
2     12125  2010         2435.8  2849447.944  1.200000e+11  320.329009
3     12125  2011         2425.2  2880486.549  1.210000e+11  373.287738
4     12125  2012         4887.6  5302853.009  2.280000e+11  359.410945
..      ...   ...            ...          ...           ...         ...
235   29510  2015         1081.1  4391209.991  5.020870e+09  215.117847
236   29510  2016         1092.7  4468866.467  5.091784e+09  221.102532
237   29510  2017         1105.3  4578251.667  5.207725e+09  228.967835
238   29510  2018         1122.2  4671352.207  5.332183e+09  238.750427
239   29510  2019         1058.1        0.000  0.000000e+00    0.000000

[240 rows x 6 columns]


In [ ]:
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataLast20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')
    
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])
    
    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 101, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 42123
1/1 [==============================] - 0s 22ms/step
For 10 neurons: MAE: 9.474531718095145, MAPE: 0.4888622544751864
1/1 [==============================] - 0s 33ms/step
For 20 neurons: MAE: 34.93139917055766, MAPE: 1.7031834215906905
1/1 [==============================] - 0s 22ms/step
For 30 neurons: MAE: 60.92351915041605, MAPE: 2.9424259053567954
1/1 [==============================] - 0s 19ms/step
For 40 neurons: MAE: 61.7598823706309, MAPE: 2.9847778052306793
1/1 [==============================] - 0s 20ms/step
For 50 neurons: MAE: 115.03762734731038, MAPE: 5.5258554569950755
1/1 [==============================] - 0s 20ms/step
For 60 neurons: MAE: 95.88543510437012, MAPE: 4.6108504926041745
1/1 [==============================] - 0s 19ms/step
For 70 neurons: MAE: 71.76468601226806, MAPE: 3.4579142973677683
1/1 [==============================] - 0s 28ms/step
For 80 neurons: MAE: 81.35239677429199, MAPE: 3.9161927166982227
1/1 [==============================] - 0

In [ ]:

#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalDataLast20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')
    
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])
    
    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 101, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsWindows9TotalTop20_HIV.csv', index=False)

Processing county 42123
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 4.011380132039388, MAPE: 0.20463350397672012
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 3.4674100240071617, MAPE: 0.16238497832964904
1/1 [==============================] - 0s 30ms/step
For 30 neurons: MAE: 3.3968718210856124, MAPE: 0.15867097585925452
1/1 [==============================] - 0s 22ms/step
For 40 neurons: MAE: 3.5975861867268875, MAPE: 0.14478933728951257
1/1 [==============================] - 0s 28ms/step
For 50 neurons: MAE: 27.70101038614909, MAPE: 1.2964004716569482
1/1 [==============================] - 0s 22ms/step
For 60 neurons: MAE: 31.772487958272297, MAPE: 1.4905331409718119
1/1 [==============================] - 0s 21ms/step
For 70 neurons: MAE: 55.046304066975914, MAPE: 2.602938299696651
1/1 [==============================] - 0s 25ms/step
For 80 neurons: MAE: 25.902695814768474, MAPE: 1.2097473318236318
1/1 [==========================

In [ ]:
#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')
    
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])
    
    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 101, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsWindows9TotalTop20_HIV.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 20ms/step
For 10 neurons: MAE: 205.26811523437496, MAPE: 0.09624902340888297
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 697.9294921874999, MAPE: 0.3264303744920553
1/1 [==============================] - 0s 19ms/step
For 30 neurons: MAE: 779.1767903645833, MAPE: 0.36439114500254455
1/1 [==============================] - 0s 20ms/step
For 40 neurons: MAE: 424.61650390625, MAPE: 0.19859805091095414


1/1 [==============================] - 0s 24ms/step
For 50 neurons: MAE: 329.9630045572917, MAPE: 0.15424931831678765


1/1 [==============================] - 0s 20ms/step
For 60 neurons: MAE: 329.4216796875, MAPE: 0.1540350917984438
1/1 [==============================] - 0s 23ms/step
For 70 neurons: MAE: 1061.2060221354166, MAPE: 0.4961631090483458
1/1 [==============================] - 0s 28ms/step
For 80 neurons: MAE: 359.9446940104167, MAPE: 0.16826143251143352
1/1 [==============================] - 0s 20ms/step
For 90 neurons: MAE: 287.579638671875, MAPE: 0.13447286443493575
1/1 [==============================] - 0s 18ms/step
For 100 neurons: MAE: 385.9444091796875, MAPE: 0.18041697338727536
Best model for county 12125 had 10 neurons with MAE: 205.26811523437496, MAPE: 0.09624902340888297
Processing county 11001
1/1 [==============================] - 0s 29ms/step
For 10 neurons: MAE: 36.04827473958343, MAPE: 0.015462645836578479
1/1 [==============================] - 0s 21ms/step
For 20 neurons: MAE: 220.5159993489583, MAPE: 0.09469118168300435
1/1 [==============================] - 0s 32ms/step
Fo

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.8581515009998384
GPU (s):
0.09358194200012804
GPU speedup over CPU: 41x


In [ ]:
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')
    
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])
    
    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 163.79196597728878, MAPE: 0.07678931176133082
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 507.8207878291608, MAPE: 0.23753288172546064
1/1 [==============================] - 0s 20ms/step
For 30 neurons: MAE: 813.4755629877249, MAPE: 0.38038265045012004
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 261.19701592822884, MAPE: 0.1221183006798499
1/1 [==============================] - 0s 31ms/step
For 50 neurons: MAE: 1417.277130933603, MAPE: 0.6626079392911588
Best model for county 12125 had 10 neurons with MAE: 163.79196597728878, MAPE: 0.07678931176133082
Processing county 11001
1/1 [==============================] - 0s 27ms/step
For 10 neurons: MAE: 35.46896869701656, MAPE: 0.015068582040157576
1/1 [==============================] - 0s 22ms/step
For 20 neurons: MAE: 328.9160525083541, MAPE: 0.1409161183154265
1/1 [======================

In [ ]:
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')
    
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])
    
    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=2, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 96.84419655412603, MAPE: 0.045403197745078026
1/1 [==============================] - 0s 21ms/step
For 20 neurons: MAE: 201.0423996279636, MAPE: 0.094034717516705
1/1 [==============================] - 0s 28ms/step
For 30 neurons: MAE: 122.29848016997191, MAPE: 0.057166867037650365
1/1 [==============================] - 0s 22ms/step
For 40 neurons: MAE: 485.5027046531438, MAPE: 0.22703420617787984
1/1 [==============================] - 0s 21ms/step
For 50 neurons: MAE: 838.363646191359, MAPE: 0.39195897641574723
Best model for county 12125 had 10 neurons with MAE: 96.84419655412603, MAPE: 0.045403197745078026
Processing county 11001
1/1 [==============================] - 0s 26ms/step
For 10 neurons: MAE: 104.74312020366399, MAPE: 0.04497163218395194
1/1 [==============================] - 0s 38ms/step
For 20 neurons: MAE: 156.10775800347332, MAPE: 0.06681165358523095
1/1 [====================

In [ ]:
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')
    
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])
    
    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 22ms/step
For 10 neurons: MAE: 189.77502958985679, MAPE: 0.08894412249901139
1/1 [==============================] - 0s 38ms/step
For 20 neurons: MAE: 82.44457727310571, MAPE: 0.03851847352292214
1/1 [==============================] - 0s 24ms/step
For 30 neurons: MAE: 406.45972281992425, MAPE: 0.19013622098789815
1/1 [==============================] - 0s 32ms/step
For 40 neurons: MAE: 1389.5218809326495, MAPE: 0.6497185313790813
1/1 [==============================] - 0s 29ms/step
For 50 neurons: MAE: 659.6192034900192, MAPE: 0.30837828025947095
Best model for county 12125 had 20 neurons with MAE: 82.44457727310571, MAPE: 0.03851847352292214
Processing county 11001
1/1 [==============================] - 0s 20ms/step
For 10 neurons: MAE: 130.30929958870033, MAPE: 0.056041998498623374
1/1 [==============================] - 0s 27ms/step
For 20 neurons: MAE: 554.5907273570696, MAPE: 0.23774349189994026
1/1 [===================

In [ ]:
#startfrom2016
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')
    
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])
    
    x_train = np.array([train[i-8:i] for i in range(8, len(train))])
    y_train = np.array([train[i, 0] for i in range(8, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2016, 2020):
            input_data = train[-8:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 8, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2016) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 36ms/step
For 10 neurons: MAE: 505.45090456143976, MAPE: 0.235363690793981
1/1 [==============================] - 0s 32ms/step
For 20 neurons: MAE: 348.084899256006, MAPE: 0.15945652977602398
1/1 [==============================] - 0s 23ms/step
For 30 neurons: MAE: 514.7906872406601, MAPE: 0.2378993746547166
1/1 [==============================] - 0s 28ms/step
For 40 neurons: MAE: 2099.8708999753003, MAPE: 0.9673382606037841
1/1 [==============================] - 0s 23ms/step
For 50 neurons: MAE: 1317.3455187976365, MAPE: 0.6071469690856702
Best model for county 12125 had 20 neurons with MAE: 348.084899256006, MAPE: 0.15945652977602398
Processing county 11001
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 243.4809454154223, MAPE: 0.10392215402719238
1/1 [==============================] - 0s 31ms/step
For 20 neurons: MAE: 402.55298034846777, MAPE: 0.1710682304182624
1/1 [===========================

## MovingWindowsTotalDataPredictionHIVPCI

In [ ]:
###HIVPCI total data
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')
    
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])
    
    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCI.csv", index=False)

Processing county 1001
1/1 [==============================] - 0s 18ms/step
For 10 neurons: MAE: 7.7623007714748455, MAPE: 0.034400089953867244
1/1 [==============================] - 0s 20ms/step
For 20 neurons: MAE: 44.654501843452415, MAPE: 0.19704653901899086
1/1 [==============================] - 0s 20ms/step
For 30 neurons: MAE: 17.810100543498987, MAPE: 0.07840572706911088
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 41.70825849771496, MAPE: 0.1835791767645789


1/1 [==============================] - 0s 20ms/step
For 50 neurons: MAE: 22.655275273323053, MAPE: 0.0998643402589572
Best model for county 1001 had 10 neurons with MAE: 7.7623007714748455, MAPE: 0.034400089953867244
Processing county 1003


1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 4.267286344369239, MAPE: 0.02504901629546774
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 10.619553724924742, MAPE: 0.06214988679186908
1/1 [==============================] - 0s 31ms/step
For 30 neurons: MAE: 13.003509281078985, MAPE: 0.07609006764787968
1/1 [==============================] - 0s 20ms/step
For 40 neurons: MAE: 22.52336186567941, MAPE: 0.13248570930597603
1/1 [==============================] - 0s 29ms/step
For 50 neurons: MAE: 2.9242241422335176, MAPE: 0.017285024362798312
Best model for county 1003 had 50 neurons with MAE: 2.9242241422335176, MAPE: 0.017285024362798312
Processing county 1005
1/1 [==============================] - 0s 19ms/step
For 10 neurons: MAE: 20.418916698296886, MAPE: 0.04598679594044123
1/1 [==============================] - 0s 38ms/step
For 20 neurons: MAE: 136.36461200714112, MAPE: 0.2954589994770622
1/1 [==============================] - 0s 21ms/

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')